<a href="https://colab.research.google.com/github/Byzed02/Byzed02/blob/main/App_Movies_Recommandations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importations des librairies

import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pickle
import pandas as pd
import streamlit as st
from streamlit import session_state as session
from flask import Flask, request, jsonify
from flask_restful import Api, Resource

app = Flask(__name__)
api = Api(app)

movies_df= pd.read_csv('movies.csv')
ratings_df= pd.read_csv('ratings.csv')

# Copie du dataframedans un nouveau dataframe
moviesWithGenres_df= movies_df.copy()

# Pour chaque ligne du dataframe, parcourir la liste de genres et placer 1 à la colonne correspondante du nouveau dataframe
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

# Remplacer les valeurs NaNpar des 0 pour indiquer qu'un film n'est pas de ce genre
moviesWithGenres_df= moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

# Supprimer une colonne ou une ligne spécifique d'un dataframe
ratings_df= ratings_df.drop('timestamp',1)

#Definition d'une fonction de recommandation

def Fonction_de_recommandation(userInput, ratings_df, moviesWithGenres_df, n=10):

    #Les notes attribuées par l'utilisateur
    inputMovies= pd.DataFrame(userInput)

    #Filtrer les films sur la base des titres
    inputId= movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

    #Fusionner de façon implicite sur la base des titre, pour avoir notre dataframe
    inputMovies= pd.merge(inputId, inputMovies)

    # Supprimer les colonnes dont nous n'avons pas besoin dans notre dataframepour libérer de la mémoire
    inputMovies= inputMovies.drop('genres', 1).drop('year', 1)

    # Filtrer les films
    userMovies= moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]

    # Réinitialisation de l'index
    userMovies= userMovies.reset_index(drop=True)

    # Supprimer les colonnes non nécessaires
    userGenreTable= userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)

    # Produit matriciel pour obtenir les poids
    userProfile= userGenreTable.transpose().dot(inputMovies['rating'])

    # Récupérons les genres de chaque film de notre dataframed'origine
    genreTable= moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

    # Et supprimons less colonnes non nécessaires
    genreTable= genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)

    # Multiplier les genres par les poids et calculer la moyenne pondérée
    recommendationTable_df= ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

    # Ordonner les recommandations par ordre décroissant
    recommendationTable_df= recommendationTable_df.sort_values(ascending=False)

    # Le résultatfinal // On va chercher des films
    Films_recommandes = movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]
    return Films_recommandes[:n]


#utilisateur
userInput= [
        {'title':'Breakfast Club, The', 'rating':5},
        {'title':'Toy Story', 'rating':2},
        {'title':'Jumanji', 'rating':3.5},
        {'title':'PulpFiction', 'rating':5},
        {'title':'Akira', 'rating':4.5}
        ]

recommanded_movies = Movies_Recommandations(userInput, ratings_df, moviesWithGenres_df, n=10)


# Definissons la fonction de lapplication

def Nom_de_application():

    st.title('Nom_de_applivation')

    #Bouton Utilisateur
    st.subheader('Entrez une note pour chaque films') # exemple de message
    userInput = []
    for i in range(5):
        Titre = st.text_input(f"Film{i+1}")
        Note = st.slider(f"Note{i+1}", min_value=0.5, max_value=5.0, step=0.5)
        if Titre and Note:
            userInput.append({"titre": Titre, "Note": Note})

    #Utiliser la fonction de recommandation
    if st.button("Valider"):
        if userInput:
            Films_recommandes = Fonction_de_recommandation(userInput, ratings_df, moviesWithGenres_df, n=10)
            st.subheader("Les Films recommandés")
            st.dataframe(Films_recommandes[['title', 'genres']])
        else:
            st.warning("Message d'erreur")



if __name__ == '__main__':
    Nom_de_application()